In [3]:
%matplotlib inline
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
pd.options.display.max_rows = 200

plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('font', size=20)



# Loading Dataset

## IOT Devices

The dataset has been extracted at `ROOT_PATH`, containing the data for each IOT device. There are 9 devices in total. There are folders for each IOT device containing `benign_traffic.csv` and further two folders for `gafgyt_attacks` and `mirai_attacks`

In [7]:
ROOT_PATH = r'C:\Users\jessm\Desktop\HonoursProject\N-BaIoT_Dataset'
IOT_DEVS = [ 'Danmini_Doorbell',
             #'Ecobee_Thermostat',
             #'Ennio_Doorbell',
             'B120N10_Baby_Mon',
             #'737E_Security_Cam',
             #'838_Security_Cam',
             #'Samsung_Webcam',
             '1002_Security_Cam',
             #'1003_Security_Cam'
           ]

## Paths to CSVs
We create a python dictionary that contains all the paths to the CSVs for the respective IOT devices

In [20]:
PATHS_DICT = {}

for i in IOT_DEVS:
    PATHS_DICT[i] = {}
    print('[' + i + ']')
    iot_dir = os.path.join(ROOT_PATH, i)
    PATHS_DICT[i]['benign'] = os.path.join(iot_dir, 'benign_traffic.csv')
    print('  ', 'benign_traffic.csv')
    attacks = [ d for d in os.listdir(iot_dir)
                   if os.path.isdir( os.path.join(iot_dir, d)) ]
    for attack in attacks:
        attack_name = attack.split('_')[0]
        
        PATHS_DICT[i][attack_name] = {}
        attack_dir = os.path.join(iot_dir, attack)
        types = [ f for f in os.listdir(attack_dir)
                   if os.path.isfile( os.path.join(attack_dir, f)) ]
        print('  ', attack_name, ': ', types)
        for t in types:
            type_name = t.split('.')[0]
            PATHS_DICT[i][attack_name][type_name] = os.path.join(attack_dir,t)

[Danmini_Doorbell]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[Ecobee_Thermostat]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[Philips_Baby_Monitor]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[737E_Security_Cam]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[838_Security_Cam]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[1002_Security_Cam]
   benign_traffic.csv
   gafgyt : 

## Loading CSVs into Pandas
We will create pandas dataframe for each botnet attack on each IOT device. For each datafram we will combine the attack traffic with the benign traffic add the following two columns:
- `traffic_type` : benign or attack (0 or 1)
- `attack_type`  : type of attack e.g, ack, scan etc.

In [11]:
import pandas as pd

nbaiot_dict = {}

for iot_dev in PATHS_DICT:
    nbaiot_dict[iot_dev] = {}
    b_df = pd.read_csv(PATHS_DICT[iot_dev]['benign'])
    b_df['traffic_type'] = 'benign'
    b_df['attack_type'] = 'benign'

    for botnet in [b for b in PATHS_DICT[iot_dev] if b != 'benign']:
        nbaiot_dict[iot_dev][botnet] = pd.DataFrame()
        nbaiot_dict[iot_dev][botnet] = pd.concat([nbaiot_dict[iot_dev][botnet], b_df])

        for attack in PATHS_DICT[iot_dev][botnet]:
            a_df = pd.read_csv(PATHS_DICT[iot_dev][botnet][attack])
            a_df['traffic_type'] = 'attack'
            a_df['attack_type'] = attack
            nbaiot_dict[iot_dev][botnet] = pd.concat([nbaiot_dict[iot_dev][botnet], a_df])


In [12]:
for k in nbaiot_dict:
    print(k)
    for b in nbaiot_dict[k]:
        print('  ',b,':','<benign+attack>')

Danmini_Doorbell
   gafgyt : <benign+attack>
   mirai : <benign+attack>
Ecobee_Thermostat
   gafgyt : <benign+attack>
   mirai : <benign+attack>
Philips_Baby_Monitor
   gafgyt : <benign+attack>
   mirai : <benign+attack>
737E_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>
838_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>
1002_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>
1003_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>


# Modeling

### Pre processing
Convert the target column `traffic_type` to integer 0 = benigh, 1 = attack 

In [13]:
for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        nbaiot = nbaiot_dict[dev][botnet]

        nbaiot.loc[ nbaiot['traffic_type']=='attack','traffic_type' ]=1
        nbaiot.loc[ nbaiot['traffic_type']=='benign','traffic_type' ]=0
        nbaiot['traffic_type'] = nbaiot['traffic_type'].astype(int)
        
        nbaiot_dict[dev][botnet] = nbaiot

### Classifiers and training functions

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [15]:
classfiers = {
    'KNN' : KNeighborsClassifier(),
    'RFR' : RandomForestClassifier(),
    'DTR' : DecisionTreeClassifier(),
    'ADB' : AdaBoostClassifier(),
    
}


## Main Loop

In [16]:
all_results = [
    ['DEVICE', 'TRAIN-BOTNET', 'TEST-BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
]

for clf_name in classfiers:
    clf = classfiers[clf_name]
    for dev in nbaiot_dict:
        if 'gafgyt' not in nbaiot_dict[dev] and 'gafgyt' not in nbaiot_dict[dev]:
            print('BOTNETS NOT FOUND, SKIPPING: ',dev)
            continue
        for trn_bot, tst_bot in [ ('gafgyt','mirai'), ('mirai','gafgyt')]:
            train_df = nbaiot_dict[dev][trn_bot]
            test_df = nbaiot_dict[dev][tst_bot]

            X_train = train_df.drop( ["attack_type", "traffic_type" ], axis=1 )
            y_train = train_df["traffic_type"]
            X_test = test_df.drop( ["attack_type", "traffic_type" ], axis=1 )
            y_test = test_df["traffic_type"]

            clf.fit(X_train, y_train)
            y_preds = clf.predict(X_test)

            clrp = classification_report(y_test, y_preds, output_dict=True)
            conf_mat = confusion_matrix(y_test, y_preds)
            fp = conf_mat[0][1]
            fn = conf_mat[1][0]
            result = [
                dev,
                trn_bot,
                tst_bot,
                clf_name,
                round(clrp['accuracy'],4),
                round(clrp['macro avg']['precision'],4),
                round(clrp['macro avg']['recall'],4),
                round(clrp['macro avg']['f1-score'],4),
                fp,
                fn
            ]
            print(result)
            all_results.append(result)


['Danmini_Doorbell', 'gafgyt', 'mirai', 'KNN', 0.5817, 0.5719, 0.7739, 0.4808, 113, 293376]
['Danmini_Doorbell', 'mirai', 'gafgyt', 'KNN', 0.3731, 0.5873, 0.6357, 0.3663, 212, 229368]
['Ecobee_Thermostat', 'gafgyt', 'mirai', 'KNN', 0.5533, 0.5256, 0.7625, 0.401, 227, 234425]
['Ecobee_Thermostat', 'mirai', 'gafgyt', 'KNN', 0.3516, 0.5278, 0.6536, 0.2998, 234, 209684]
['Philips_Baby_Monitor', 'gafgyt', 'mirai', 'KNN', 0.5988, 0.6785, 0.7417, 0.5892, 55, 315246]
['Philips_Baby_Monitor', 'mirai', 'gafgyt', 'KNN', 0.5291, 0.6987, 0.6288, 0.514, 3044, 226720]
['737E_Security_Cam', 'gafgyt', 'mirai', 'KNN', 0.6729, 0.6377, 0.8124, 0.6013, 115, 162822]
['737E_Security_Cam', 'mirai', 'gafgyt', 'KNN', 0.4402, 0.607, 0.6634, 0.4311, 613, 218958]
['838_Security_Cam', 'gafgyt', 'mirai', 'KNN', 0.6655, 0.6784, 0.7934, 0.634, 246, 176339]
['838_Security_Cam', 'mirai', 'gafgyt', 'KNN', 0.4753, 0.6514, 0.6502, 0.4753, 1102, 212744]
['1002_Security_Cam', 'gafgyt', 'mirai', 'KNN', 0.4983, 0.5708, 0.7253,

In [17]:
res_df =pd.DataFrame(all_results, index=None)

res_df, res_df.columns = res_df[1:] , res_df.iloc[0]

res_df

,DEVICE,TRAIN-BOTNET,TEST-BOTNET,CLASSIFIER,ACCURACY,PRECISION,RECALL,F1-SCORE,FALSE-P,FALSE-N
1,Danmini_Doorbell,gafgyt,mirai,KNN,0.5817,0.5719,0.7739,0.4808,113,293376
2,Danmini_Doorbell,mirai,gafgyt,KNN,0.3731,0.5873,0.6357,0.3663,212,229368
3,Ecobee_Thermostat,gafgyt,mirai,KNN,0.5533,0.5256,0.7625,0.401,227,234425
4,Ecobee_Thermostat,mirai,gafgyt,KNN,0.3516,0.5278,0.6536,0.2998,234,209684
5,Philips_Baby_Monitor,gafgyt,mirai,KNN,0.5988,0.6785,0.7417,0.5892,55,315246
6,Philips_Baby_Monitor,mirai,gafgyt,KNN,0.5291,0.6987,0.6288,0.514,3044,226720
7,737E_Security_Cam,gafgyt,mirai,KNN,0.6729,0.6377,0.8124,0.6013,115,162822
8,737E_Security_Cam,mirai,gafgyt,KNN,0.4402,0.607,0.6634,0.4311,613,218958
9,838_Security_Cam,gafgyt,mirai,KNN,0.6655,0.6784,0.7934,0.634,246,176339
10,838_Security_Cam,mirai,gafgyt,KNN,0.4753,0.6514,0.6502,0.4753,1102,212744


In [18]:
# Write to csv
res_csv = res_df.to_csv(index=False)

print(res_csv, file=open('05_results.csv', 'w'))


### Load results from CSV

In [23]:
res_df = pd.read_csv('05_results.csv')

#res_df = res_df[['DEVICE', 'BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'FALSE-P', 'FALSE-N' ]]

res_df

,DEVICE,TRAIN-BOTNET,TEST-BOTNET,CLASSIFIER,ACCURACY,PRECISION,RECALL,F1-SCORE,FALSE-P,FALSE-N
0,Danmini_Doorbell,gafgyt,mirai,KNN,0.5817,0.5719,0.7739,0.4808,113,293376
1,Danmini_Doorbell,mirai,gafgyt,KNN,0.3731,0.5873,0.6357,0.3663,212,229368
2,Ecobee_Thermostat,gafgyt,mirai,KNN,0.5533,0.5256,0.7625,0.4010,227,234425
3,Ecobee_Thermostat,mirai,gafgyt,KNN,0.3516,0.5278,0.6536,0.2998,234,209684
4,Philips_Baby_Monitor,gafgyt,mirai,KNN,0.5988,0.6785,0.7417,0.5892,55,315246
5,Philips_Baby_Monitor,mirai,gafgyt,KNN,0.5291,0.6987,0.6288,0.5140,3044,226720
6,737E_Security_Cam,gafgyt,mirai,KNN,0.6729,0.6377,0.8124,0.6013,115,162822
7,737E_Security_Cam,mirai,gafgyt,KNN,0.4402,0.6070,0.6634,0.4311,613,218958
8,838_Security_Cam,gafgyt,mirai,KNN,0.6655,0.6784,0.7934,0.6340,246,176339
9,838_Security_Cam,mirai,gafgyt,KNN,0.4753,0.6514,0.6502,0.4753,1102,212744


In [24]:
#do classification report

In [28]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [30]:
from sklearn.metrics import classification_report
from tabulate import tabulate

# Load results from CSV
res_df = pd.read_csv('05_results.csv')

# Filter the relevant columns
filtered_df = res_df[['TEST-BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N']]

# Group the data by botnet and classifier
grouped = filtered_df.groupby(['TEST-BOTNET', 'CLASSIFIER'])

# Store classification report results in a list
classification_reports = []

# Iterate through the grouped data and generate classification report
for (botnet, clf), group in grouped:
    avg_accuracy = group['ACCURACY'].mean()
    avg_precision = group['PRECISION'].mean()
    avg_recall = group['RECALL'].mean()
    avg_f1_score = group['F1-SCORE'].mean()
    avg_false_positive = group['FALSE-P'].mean()
    avg_false_negative = group['FALSE-N'].mean()

    report = f"Botnet: {botnet}, Classifier: {clf}\n"
    report += tabulate(
        [[avg_accuracy, avg_precision, avg_recall, avg_f1_score, avg_false_positive, avg_false_negative]],
        headers=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'False Positives', 'False Negatives'],
        tablefmt='grid'
    )
    classification_reports.append(report)

# Print the classification reports
for report in classification_reports:
    print(report)
    print("\n\n")



Botnet: gafgyt, Classifier: ADB
+------------+-------------+----------+------------+-------------------+-------------------+
|   Accuracy |   Precision |   Recall |   F1-Score |   False Positives |   False Negatives |
+============+=============+==========+============+===================+===================+
|   0.455029 |    0.609886 | 0.674014 |   0.429843 |                 0 |            204728 |
+------------+-------------+----------+------------+-------------------+-------------------+



Botnet: gafgyt, Classifier: DTR
+------------+-------------+----------+------------+-------------------+-------------------+
|   Accuracy |   Precision |   Recall |   F1-Score |   False Positives |   False Negatives |
+============+=============+==========+============+===================+===================+
|   0.458129 |      0.6102 | 0.675686 |   0.432271 |                 0 |            203704 |
+------------+-------------+----------+------------+-------------------+-------------------+





In [36]:

# Load results from CSV
res_df = pd.read_csv('05_results.csv')

# Filter the relevant columns
filtered_df = res_df[['TEST-BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N']]

# Group the data by botnet and classifier
grouped = filtered_df.groupby(['TEST-BOTNET', 'CLASSIFIER'])

# Store classification report results in a dictionary for each botnet
botnet_reports = {}

# Iterate through the grouped data and generate classification reports
for (botnet, clf), group in grouped:
    avg_metrics = group.mean()
    metrics_names = ['ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N']
    avg_values = avg_metrics[metrics_names].values

    if botnet not in botnet_reports:
        botnet_reports[botnet] = []

    botnet_reports[botnet].append([clf] + avg_values.tolist())

# Print the classification reports for each botnet
for botnet, reports in botnet_reports.items():
    headers = ['CLASSIFIER'] + metrics_names
    table = tabulate(reports, headers=headers, tablefmt='grid')
    
    print(f"Botnet: {botnet}")
    print(table)
    print("\n")


Botnet: gafgyt
+--------------+------------+-------------+----------+------------+-----------+-----------+
| CLASSIFIER   |   ACCURACY |   PRECISION |   RECALL |   F1-SCORE |   FALSE-P |   FALSE-N |
+==============+============+=============+==========+============+===========+===========+
| ADB          |   0.455029 |    0.609886 | 0.674014 |   0.429843 |     0     |    204728 |
+--------------+------------+-------------+----------+------------+-----------+-----------+
| DTR          |   0.458129 |    0.6102   | 0.675686 |   0.432271 |     0     |    203704 |
+--------------+------------+-------------+----------+------------+-----------+-----------+
| KNN          |   0.418743 |    0.600186 | 0.648671 |   0.399271 |   806.857 |    217341 |
+--------------+------------+-------------+----------+------------+-----------+-----------+
| RFR          |   0.464843 |    0.612014 | 0.680829 |   0.439957 |     0     |    200484 |
+--------------+------------+-------------+----------+-----------

C:\Users\jessm\AppData\Local\Temp\ipykernel_24216\1550177976.py:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_metrics = group.mean()
C:\Users\jessm\AppData\Local\Temp\ipykernel_24216\1550177976.py:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_metrics = group.mean()
C:\Users\jessm\AppData\Local\Temp\ipykernel_24216\1550177976.py:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated